In [1]:
# Para trabajar con ficheros
# -----------------------------------------------------------------------
import os

import dotenv
from dotenv import load_dotenv
load_dotenv()
# Para trabajar con los DataFrames
# -----------------------------------------------------------------------
import pandas as pd

# Para poner barras progreso
# -----------------------------------------------------------------------
from tqdm import tqdm

# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames

from datetime import datetime

import psycopg2
from psycopg2 import sql
# Ignorar warings
# -----------------------------------------------------------------------
import warnings
warnings.filterwarnings("ignore")

In [2]:
df_economico=pd.read_csv("datos/datos_agregados/economico_agr.csv", index_col= "Unnamed: 0")
df_poblacional=pd.read_csv("datos/datos_agregados/poblacional_agr.csv", index_col= "Unnamed: 0")
df_todo_demanda=pd.read_csv("datos/datos_agregados/demanda_agr.csv", index_col= "Unnamed: 0")
df_todo_oferta=pd.read_csv("datos/datos_agregados/oferta_agr.csv", index_col= "Unnamed: 0")

In [3]:
df_todo_oferta.head(3)

,value,percentage,type,cod_ccaa,ccaa,month,year
0,25288.605,0.029517,Hidráulica,4,Andalucía,1,2019
1,33933.350,0.030779,Hidráulica,4,Andalucía,2,2019
2,35913.000,0.029645,Hidráulica,4,Andalucía,3,2019


In [4]:
df_todo_oferta.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2909 entries, 0 to 2908
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   value       2909 non-null   float64
 1   percentage  2909 non-null   float64
 2   type        2909 non-null   object 
 3   cod_ccaa    2909 non-null   int64  
 4   ccaa        2909 non-null   object 
 5   month       2909 non-null   int64  
 6   year        2909 non-null   int64  
dtypes: float64(2), int64(3), object(2)
memory usage: 181.8+ KB


In [5]:
df_todo_demanda.head(3)

,value,percentage,cod_ccaa,ccaa,month,year
0,3668893.328,1,4,Andalucía,1,2019
1,3152278.091,1,4,Andalucía,2,2019
2,3216356.360,1,4,Andalucía,3,2019


In [6]:
df_todo_demanda.info()

<class 'pandas.core.frame.DataFrame'>
Index: 684 entries, 0 to 683
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   value       684 non-null    float64
 1   percentage  684 non-null    int64  
 2   cod_ccaa    684 non-null    int64  
 3   ccaa        684 non-null    object 
 4   month       684 non-null    int64  
 5   year        684 non-null    int64  
dtypes: float64(1), int64(4), object(1)
memory usage: 37.4+ KB


In [7]:
df_poblacional.head(3)

,Edad (3 grupos de edad),Españoles/Extranjeros,Sexo,Año,Total,cod_prov,provincias
0,0-15 años,Españoles,Hombres,2021,27304.0,2,Albacete
1,0-15 años,Españoles,Hombres,2020,27979.0,2,Albacete
2,0-15 años,Españoles,Hombres,2019,28513.0,2,Albacete


In [8]:
df_poblacional.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1872 entries, 0 to 1871
Data columns (total 7 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Edad (3 grupos de edad)  1872 non-null   object 
 1   Españoles/Extranjeros    1872 non-null   object 
 2   Sexo                     1872 non-null   object 
 3   Año                      1872 non-null   int64  
 4   Total                    1872 non-null   float64
 5   cod_prov                 1872 non-null   int64  
 6   provincias               1872 non-null   object 
dtypes: float64(1), int64(2), object(4)
memory usage: 117.0+ KB


In [9]:
df_economico.head(3)

,periodo,Total,cod_prov,provincias,cod_actividad,ramas de actividad
0,2021,884324,2,Albacete,1,"Agricultura, ganadería, silvicultura y pesca"
1,2020,768976,2,Albacete,1,"Agricultura, ganadería, silvicultura y pesca"
2,2019,791464,2,Albacete,1,"Agricultura, ganadería, silvicultura y pesca"


In [10]:
df_economico.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1092 entries, 0 to 1091
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   periodo             1092 non-null   int64 
 1   Total               1092 non-null   int64 
 2   cod_prov            1092 non-null   int64 
 3   provincias          1092 non-null   object
 4   cod_actividad       1092 non-null   int64 
 5   ramas de actividad  1092 non-null   object
dtypes: int64(4), object(2)
memory usage: 59.7+ KB


EMPIEZO CREANDO LAS TABLAS SIN FK

In [11]:
def crear_id(df_anti,nombre_col_antigua,nombre_col_nueva,nombre_col_id_nuevo):
    anio= pd.DataFrame()
    anio[nombre_col_nueva]=df_anti[nombre_col_antigua].unique()
    anio.index=anio.index+1
    df_nuevo=anio.reset_index().rename(columns={'index': nombre_col_id_nuevo})
    return df_nuevo


In [12]:
df_anio=crear_id(df_economico,"periodo","anio","id_anio")

In [13]:
df_anio

,id_anio,anio
0,1,2021
1,2,2020
2,3,2019


In [14]:
df_actividad=pd.DataFrame()
df_actividad["actividad"] =df_economico["ramas de actividad"].unique()
df_actividad["id_actividad"]= df_economico["cod_actividad"].unique()

In [15]:
df_actividad

,actividad,id_actividad
0,"Agricultura, ganadería, silvicultura y pesca",1
1,"Industrias extractivas, industria manufacture...",2
2,- De las cuales: Industria manufacturera,3
3,Construcción,7
4,"Comercio al por mayor y al por menor, reparac...",4
5,"Actividades financieras y de seguros, activid...",5
6,"Administración pública y defensa, seguridad s...",6


In [16]:
df_nacionalidad=crear_id(df_poblacional,"Españoles/Extranjeros","nacionalidad","id_nacionalidad")

In [17]:
df_nacionalidad

,id_nacionalidad,nacionalidad
0,1,Españoles
1,2,Extranjeros


In [18]:
df_sexo=crear_id(df_poblacional,"Sexo","sexo","id_sexo")

In [19]:
df_sexo

,id_sexo,sexo
0,1,Hombres
1,2,Mujeres


In [20]:
df_edad=crear_id(df_poblacional,"Edad (3 grupos de edad)","rango_edad","id_rango_edad")

In [21]:
df_edad

,id_rango_edad,rango_edad
0,1,0-15 años
1,2,16-64 años
2,3,65 y más


In [22]:
df_type=crear_id(df_todo_oferta,"type","type","id_type")

In [23]:
df_type

,id_type,type
0,1,Hidráulica
1,2,Eólica
2,3,Solar fotovoltaica
3,4,Solar térmica
4,5,Otras renovables
5,6,Hidroeólica
6,7,Residuos renovables


In [24]:
df_CCAA=pd.DataFrame()
df_CCAA["ccaa"] =df_todo_demanda["ccaa"].unique()
df_CCAA["id_ccaa"]= df_todo_demanda["cod_ccaa"].unique()

In [25]:
df_CCAA.head(3)

,ccaa,id_ccaa
0,Andalucía,4
1,Aragón,5
2,Canarias,8742


In [26]:
df_provincia=pd.DataFrame()
df_provincia["provincia"] =df_economico["provincias"].unique()
df_provincia["id_provincia"]= df_economico["cod_prov"].unique()

In [27]:
df_provincia.head(3)

,provincia,id_provincia
0,Albacete,2
1,Alicante/Alacant,3
2,Almería,4


EMPEZAMOS CON LAS TABLAS HEAVIES

DEMANDA (quitamos ccaa, cambiamos nombre de cod_ccaa a id_ccaa, mapeamos y metemos el id_anio)

In [28]:
df_todo_demanda.head()

,value,percentage,cod_ccaa,ccaa,month,year
0,3668893.328,1,4,Andalucía,1,2019
1,3152278.091,1,4,Andalucía,2,2019
2,3216356.360,1,4,Andalucía,3,2019
3,3029639.973,1,4,Andalucía,4,2019
4,3154123.467,1,4,Andalucía,5,2019


In [29]:
df_todo_demanda.index=df_todo_demanda.index+1
df_todo_demanda=df_todo_demanda.reset_index().rename(columns={'index': "id_demanda"})

In [30]:
df_todo_demanda=df_todo_demanda.rename(columns={'cod_ccaa': "id_ccaa"})
df_todo_demanda.drop(columns="ccaa", inplace=True)

In [31]:
df_todo_demanda.head()

,id_demanda,value,percentage,id_ccaa,month,year
0,1,3668893.328,1,4,1,2019
1,2,3152278.091,1,4,2,2019
2,3,3216356.360,1,4,3,2019
3,4,3029639.973,1,4,4,2019
4,5,3154123.467,1,4,5,2019


In [32]:

dicc_anio = df_anio.set_index('anio')['id_anio'].to_dict()
dicc_anio

{2021: 1, 2020: 2, 2019: 3}

In [33]:

df_todo_demanda['id_anio'] = df_todo_demanda['year'].map(dicc_anio)


In [34]:
df_todo_demanda.drop(columns="year", inplace=True)

In [35]:
df_todo_demanda.head()

,id_demanda,value,percentage,id_ccaa,month,id_anio
0,1,3668893.328,1,4,1,3
1,2,3152278.091,1,4,2,3
2,3,3216356.360,1,4,3,3
3,4,3029639.973,1,4,4,3
4,5,3154123.467,1,4,5,3


OFERTA

In [36]:
df_todo_oferta.head()

,value,percentage,type,cod_ccaa,ccaa,month,year
0,25288.605,0.029517,Hidráulica,4,Andalucía,1,2019
1,33933.350,0.030779,Hidráulica,4,Andalucía,2,2019
2,35913.000,0.029645,Hidráulica,4,Andalucía,3,2019
3,44050.848,0.041340,Hidráulica,4,Andalucía,4,2019
4,88535.452,0.068497,Hidráulica,4,Andalucía,5,2019


In [37]:
df_todo_oferta.index=df_todo_oferta.index+1
df_todo_oferta=df_todo_oferta.reset_index().rename(columns={'index': "id_oferta"})

In [38]:
df_todo_oferta=df_todo_oferta.rename(columns={'cod_ccaa': "id_ccaa"})
df_todo_oferta.drop(columns="ccaa", inplace=True)

In [39]:
df_todo_oferta.head()

,id_oferta,value,percentage,type,id_ccaa,month,year
0,1,25288.605,0.029517,Hidráulica,4,1,2019
1,2,33933.350,0.030779,Hidráulica,4,2,2019
2,3,35913.000,0.029645,Hidráulica,4,3,2019
3,4,44050.848,0.041340,Hidráulica,4,4,2019
4,5,88535.452,0.068497,Hidráulica,4,5,2019


In [40]:

df_todo_oferta['id_anio'] = df_todo_oferta['year'].map(dicc_anio)


In [41]:
dicc_type = df_type.set_index('type')['id_type'].to_dict()
dicc_type

{'Hidráulica': 1,
 'Eólica': 2,
 'Solar fotovoltaica': 3,
 'Solar térmica': 4,
 'Otras renovables': 5,
 'Hidroeólica': 6,
 'Residuos renovables': 7}

In [42]:
df_todo_oferta['id_type'] = df_todo_oferta['type'].map(dicc_type)

In [43]:
df_todo_oferta.drop(columns="year", inplace=True)
df_todo_oferta.drop(columns="type", inplace=True)

In [44]:
df_todo_oferta.head()

,id_oferta,value,percentage,id_ccaa,month,id_anio,id_type
0,1,25288.605,0.029517,4,1,3,1
1,2,33933.350,0.030779,4,2,3,1
2,3,35913.000,0.029645,4,3,3,1
3,4,44050.848,0.041340,4,4,3,1
4,5,88535.452,0.068497,4,5,3,1


ECONOMICO

In [45]:
df_economico.head()

,periodo,Total,cod_prov,provincias,cod_actividad,ramas de actividad
0,2021,884324,2,Albacete,1,"Agricultura, ganadería, silvicultura y pesca"
1,2020,768976,2,Albacete,1,"Agricultura, ganadería, silvicultura y pesca"
2,2019,791464,2,Albacete,1,"Agricultura, ganadería, silvicultura y pesca"
3,2021,1397006,2,Albacete,2,"Industrias extractivas, industria manufacture..."
4,2020,1194438,2,Albacete,2,"Industrias extractivas, industria manufacture..."


In [46]:
df_economico=df_economico.rename(columns={'cod_actividad': "id_actividad"})
df_economico=df_economico.rename(columns={'cod_prov': "id_provincia"})
df_economico.drop(columns="ramas de actividad", inplace=True)
df_economico.drop(columns="provincias", inplace=True)

In [47]:
df_economico.index=df_economico.index+1
df_economico=df_economico.reset_index().rename(columns={'index': "id_economico"})

In [48]:
df_economico.head()

,id_economico,periodo,Total,id_provincia,id_actividad
0,1,2021,884324,2,1
1,2,2020,768976,2,1
2,3,2019,791464,2,1
3,4,2021,1397006,2,2
4,5,2020,1194438,2,2


In [49]:

df_economico['id_anio'] = df_economico['periodo'].map(dicc_anio)


In [50]:
df_economico.drop(columns="periodo", inplace=True)


In [51]:
df_economico.head()

,id_economico,Total,id_provincia,id_actividad,id_anio
0,1,884324,2,1,1
1,2,768976,2,1,2
2,3,791464,2,1,3
3,4,1397006,2,2,1
4,5,1194438,2,2,2


POBLACIONAL

In [52]:
df_poblacional.head()

,Edad (3 grupos de edad),Españoles/Extranjeros,Sexo,Año,Total,cod_prov,provincias
0,0-15 años,Españoles,Hombres,2021,27304.0,2,Albacete
1,0-15 años,Españoles,Hombres,2020,27979.0,2,Albacete
2,0-15 años,Españoles,Hombres,2019,28513.0,2,Albacete
3,0-15 años,Españoles,Mujeres,2021,25547.0,2,Albacete
4,0-15 años,Españoles,Mujeres,2020,25953.0,2,Albacete


In [53]:
df_poblacional.index=df_poblacional.index+1
df_poblacional=df_poblacional.reset_index().rename(columns={'index': "id_poblacion"})

In [54]:
df_poblacional=df_poblacional.rename(columns={'cod_prov': "id_provincia"})
df_poblacional.drop(columns="provincias", inplace=True)

In [55]:
df_poblacional.head()

,id_poblacion,Edad (3 grupos de edad),Españoles/Extranjeros,Sexo,Año,Total,id_provincia
0,1,0-15 años,Españoles,Hombres,2021,27304.0,2
1,2,0-15 años,Españoles,Hombres,2020,27979.0,2
2,3,0-15 años,Españoles,Hombres,2019,28513.0,2
3,4,0-15 años,Españoles,Mujeres,2021,25547.0,2
4,5,0-15 años,Españoles,Mujeres,2020,25953.0,2


In [56]:
dicc_edad = df_edad.set_index('rango_edad')['id_rango_edad'].to_dict()
dicc_nac = df_nacionalidad.set_index('nacionalidad')['id_nacionalidad'].to_dict()
dicc_sexo = df_sexo.set_index('sexo')['id_sexo'].to_dict()


In [57]:
df_poblacional['id_rango_edad'] = df_poblacional['Edad (3 grupos de edad)'].map(dicc_edad)
df_poblacional['id_nacionalidad'] = df_poblacional['Españoles/Extranjeros'].map(dicc_nac)
df_poblacional['id_sexo'] = df_poblacional['Sexo'].map(dicc_sexo)
df_poblacional['id_anio'] = df_poblacional['Año'].map(dicc_anio)


In [58]:
df_poblacional.drop(columns="Edad (3 grupos de edad)", inplace=True)
df_poblacional.drop(columns="Españoles/Extranjeros", inplace=True)
df_poblacional.drop(columns="Sexo", inplace=True)
df_poblacional.drop(columns="Año", inplace=True)

In [59]:
df_poblacional.head()

,id_poblacion,Total,id_provincia,id_rango_edad,id_nacionalidad,id_sexo,id_anio
0,1,27304.0,2,1,1,1,1
1,2,27979.0,2,1,1,1,2
2,3,28513.0,2,1,1,1,3
3,4,25547.0,2,1,1,2,1
4,5,25953.0,2,1,1,2,2


CREAR LA BASE DE DATOS EN SQL

In [60]:
key= os.getenv("clave")

In [147]:

conexion = psycopg2.connect(
    host="localhost",       
    user="postgres",      
    password=key, 
    port="5432"            
)


conexion.autocommit = True  
cursor = conexion.cursor()

nombre_base_datos = "Sistema_energetico"

try:
    cursor.execute(sql.SQL("CREATE DATABASE {}").format(sql.Identifier(nombre_base_datos)))
except psycopg2.Error as e:
    print("Error de tipo:", e)





Error de tipo: la base de datos «Sistema_energetico» ya existe



In [148]:
try:
    conexion= psycopg2.connect(
        database= "Sistema_energetico",
        user= "postgres",
        password= key,
        host= "localhost",
        port= "5432"
    )
except:
    print(f"Ocurrió el error {e}")

CREAR TABLAS DE LA BASE DE DATOS EN SQL

ACTIVIDAD

In [63]:
df_actividad.head(2)

,actividad,id_actividad
0,"Agricultura, ganadería, silvicultura y pesca",1
1,"Industrias extractivas, industria manufacture...",2


In [64]:
cursor= conexion.cursor()
query_creacion= """
CREATE TABLE IF NOT EXISTS actividades(
    actividad VARCHAR(500),
    id_actividad INT PRIMARY KEY
);
"""
cursor.execute(query_creacion)   
conexion.commit()

In [65]:
lista_de_tuplas=[]
for fila in df_actividad.values:
    lista_de_tuplas.append(tuple(fila))


In [66]:
     
query_insercion= "INSERT INTO actividades (actividad, id_actividad) VALUES (%s, %s);"     

cursor.executemany(query_insercion, lista_de_tuplas)          
conexion.commit()

NACIONALIDAD

In [67]:
df_nacionalidad.head(2)

,id_nacionalidad,nacionalidad
0,1,Españoles
1,2,Extranjeros


In [68]:
cursor= conexion.cursor()
query_creacion= """
CREATE TABLE IF NOT EXISTS nacionalidades(
    id_nacionalidad INT PRIMARY KEY,
    nacionalidad VARCHAR(100)
);
"""
cursor.execute(query_creacion)   
conexion.commit()

In [69]:
lista_de_tuplas=[]
for fila in df_nacionalidad.values:
    lista_de_tuplas.append(tuple(fila))


In [70]:
     
query_insercion= "INSERT INTO nacionalidades (id_nacionalidad, nacionalidad) VALUES (%s, %s);"     

cursor.executemany(query_insercion, lista_de_tuplas)          
conexion.commit()

Sexo

In [71]:
df_sexo.head(2)

,id_sexo,sexo
0,1,Hombres
1,2,Mujeres


In [72]:
cursor= conexion.cursor()
query_creacion= """
CREATE TABLE IF NOT EXISTS sexos(
    id_sexo INT PRIMARY KEY,
    sexo VARCHAR(100)
);
"""
cursor.execute(query_creacion)   
conexion.commit()

In [73]:
lista_de_tuplas=[]
for fila in df_sexo.values:
    lista_de_tuplas.append(tuple(fila))


In [74]:
     
query_insercion= "INSERT INTO sexos (id_sexo, sexo) VALUES (%s, %s);"     

cursor.executemany(query_insercion, lista_de_tuplas)          
conexion.commit()

Edad

In [75]:
df_edad.head(2)

,id_rango_edad,rango_edad
0,1,0-15 años
1,2,16-64 años


In [76]:
cursor= conexion.cursor()
query_creacion= """
CREATE TABLE IF NOT EXISTS rango_edades(
    id_rango_edad INT PRIMARY KEY,
    rango_edad VARCHAR(100)
);
"""
cursor.execute(query_creacion)   
conexion.commit()

In [77]:
lista_de_tuplas=[]
for fila in df_edad.values:
    lista_de_tuplas.append(tuple(fila))


In [78]:
     
query_insercion= "INSERT INTO rango_edades (id_rango_edad, rango_edad) VALUES (%s, %s);"     

cursor.executemany(query_insercion, lista_de_tuplas)          
conexion.commit()

Type

In [79]:
df_type.head(2)

,id_type,type
0,1,Hidráulica
1,2,Eólica


In [80]:
cursor= conexion.cursor()
query_creacion= """
CREATE TABLE IF NOT EXISTS types(
    id_type INT PRIMARY KEY,
    type VARCHAR(100)
);
"""
cursor.execute(query_creacion)   
conexion.commit()

In [81]:
lista_de_tuplas=[]
for fila in df_type.values:
    lista_de_tuplas.append(tuple(fila))


In [82]:
     
query_insercion= "INSERT INTO types (id_type, type) VALUES (%s, %s);"     

cursor.executemany(query_insercion, lista_de_tuplas)          
conexion.commit()

Provincias

In [83]:
df_provincia.head(2)

,provincia,id_provincia
0,Albacete,2
1,Alicante/Alacant,3


In [84]:
cursor= conexion.cursor()
query_creacion= """
CREATE TABLE IF NOT EXISTS provincias(
    provincia VARCHAR(100),
    id_provincia INT PRIMARY KEY
    
);
"""
cursor.execute(query_creacion)   
conexion.commit()

In [85]:
lista_de_tuplas=[]
for fila in df_provincia.values:
    lista_de_tuplas.append(tuple(fila))


In [86]:
     
query_insercion= "INSERT INTO provincias (provincia, id_provincia) VALUES (%s, %s);"     

cursor.executemany(query_insercion, lista_de_tuplas)          
conexion.commit()

Comunidades

In [87]:
df_CCAA.head(2)

,ccaa,id_ccaa
0,Andalucía,4
1,Aragón,5


In [88]:
cursor= conexion.cursor()
query_creacion= """
CREATE TABLE IF NOT EXISTS comunidades(
    ccaa VARCHAR(100),
    id_ccaa INT PRIMARY KEY
    
);
"""
cursor.execute(query_creacion)   
conexion.commit()

In [89]:
lista_de_tuplas=[]
for fila in df_CCAA.values:
    lista_de_tuplas.append(tuple(fila))


In [90]:
     
query_insercion= "INSERT INTO comunidades (ccaa, id_ccaa) VALUES (%s, %s);"     

cursor.executemany(query_insercion, lista_de_tuplas)          
conexion.commit()

AÑOS

In [91]:
df_anio.head(2)

,id_anio,anio
0,1,2021
1,2,2020


In [92]:
cursor= conexion.cursor()
query_creacion= """
CREATE TABLE IF NOT EXISTS anios(
    id_anio INT PRIMARY KEY,
    anio INT NOT NULL
    
    
);
"""
cursor.execute(query_creacion)   
conexion.commit()

In [93]:
lista_de_tuplas=[]
for fila in df_anio.values:
    lista_de_tuplas.append(tuple(fila))
tuplas_float = [tuple(float(valor) for valor in tupla) for tupla in lista_de_tuplas]

In [94]:
     
query_insercion= "INSERT INTO anios (id_anio, anio) VALUES (%s, %s);"     

cursor.executemany(query_insercion, tuplas_float)          
conexion.commit()

DEMANDA

In [95]:
df_todo_demanda.head(2)

,id_demanda,value,percentage,id_ccaa,month,id_anio
0,1,3668893.328,1,4,1,3
1,2,3152278.091,1,4,2,3


In [96]:
cursor= conexion.cursor()
query_creacion= """
CREATE TABLE IF NOT EXISTS demanda_energetica(
    id_demanda INT PRIMARY KEY,
    value NUMERIC,
    percentage NUMERIC,
    id_ccaa INT,
    month INT,
    id_anio INT,
    FOREIGN KEY (id_ccaa) REFERENCES comunidades(id_ccaa) ON DELETE RESTRICT ON UPDATE CASCADE,
    FOREIGN KEY (id_anio) REFERENCES anios(id_anio) ON DELETE RESTRICT ON UPDATE CASCADE
    
    
);
"""
cursor.execute(query_creacion)   
conexion.commit()

In [97]:
lista_de_tuplas=[]
for fila in df_todo_demanda.values:
    lista_de_tuplas.append(tuple(fila))


In [98]:
lista_de_tuplas = [(int(x),float(y),int(z),int(a),int(b),int(c))for x,y,z,a,b,c in lista_de_tuplas]


In [99]:
     
query_insercion= "INSERT INTO demanda_energetica (id_demanda,value,percentage,id_ccaa,month,id_anio) VALUES (%s, %s, %s, %s, %s, %s);"     

cursor.executemany(query_insercion, lista_de_tuplas)          
conexion.commit()

OFERTA

In [100]:
df_todo_oferta.head(2)

,id_oferta,value,percentage,id_ccaa,month,id_anio,id_type
0,1,25288.605,0.029517,4,1,3,1
1,2,33933.350,0.030779,4,2,3,1


In [121]:
cursor= conexion.cursor()
query_creacion= """
CREATE TABLE IF NOT EXISTS oferta_energetica(
    id_oferta INT PRIMARY KEY,
    value NUMERIC,
    percentage NUMERIC,
    id_ccaa INT,
    month INT,
    id_anio INT,
    id_type INT,
    FOREIGN KEY (id_ccaa) REFERENCES comunidades(id_ccaa) ON DELETE RESTRICT ON UPDATE CASCADE,
    FOREIGN KEY (id_anio) REFERENCES anios(id_anio) ON DELETE RESTRICT ON UPDATE CASCADE,
    FOREIGN KEY (id_type) REFERENCES types(id_type) ON DELETE RESTRICT ON UPDATE CASCADE
    
    
);
"""
cursor.execute(query_creacion)   
conexion.commit()

In [122]:
lista_de_tuplas=[]
for fila in df_todo_oferta.values:
    lista_de_tuplas.append(tuple(fila))


In [123]:
lista_de_tuplas = [(int(x),float(y),float(z),int(a),int(b),int(c),int(w))for x,y,z,a,b,c,w in lista_de_tuplas]


In [125]:
     
query_insercion= "INSERT INTO oferta_energetica (id_oferta,value,percentage,id_ccaa,month,id_anio,id_type) VALUES (%s, %s, %s, %s, %s, %s, %s);"     

cursor.executemany(query_insercion, lista_de_tuplas)          
conexion.commit()

ECONOMICO

In [126]:
df_economico.head(2)

,id_economico,Total,id_provincia,id_actividad,id_anio
0,1,884324,2,1,1
1,2,768976,2,1,2


In [149]:
cursor= conexion.cursor()
query_creacion= """
CREATE TABLE IF NOT EXISTS economicos(
    id_economico INT PRIMARY KEY,
    Total NUMERIC,
    id_provincia INT,
    id_actividad INT,
    id_anio INT,
    FOREIGN KEY (id_provincia) REFERENCES provincias(id_provincia) ON DELETE RESTRICT ON UPDATE CASCADE,
    FOREIGN KEY (id_actividad) REFERENCES actividades(id_actividad) ON DELETE RESTRICT ON UPDATE CASCADE,
    FOREIGN KEY (id_anio) REFERENCES anios(id_anio) ON DELETE RESTRICT ON UPDATE CASCADE
    
    
);
"""
cursor.execute(query_creacion)   
conexion.commit()

In [150]:
lista_de_tuplas=[]
for fila in df_economico.values:
    lista_de_tuplas.append(tuple(fila))


In [151]:
lista_de_tuplas = [(int(x),int(a),int(b),int(c),int(w))for x,a,b,c,w in lista_de_tuplas]


In [152]:
     
query_insercion= "INSERT INTO economicos (id_economico,Total,id_provincia,id_actividad,id_anio) VALUES (%s, %s, %s, %s, %s);"     

cursor.executemany(query_insercion, lista_de_tuplas)          
conexion.commit()

POBLACIONAL

In [153]:
df_poblacional.head(2)

,id_poblacion,Total,id_provincia,id_rango_edad,id_nacionalidad,id_sexo,id_anio
0,1,27304.0,2,1,1,1,1
1,2,27979.0,2,1,1,1,2


In [154]:
cursor= conexion.cursor()
query_creacion= """
CREATE TABLE IF NOT EXISTS poblacionales(
    id_poblacion INT PRIMARY KEY,
    Total NUMERIC,
    id_provincia INT,
    id_rango_edad INT,
    id_nacionalidad INT,
    id_sexo INT,
    id_anio INT,
    FOREIGN KEY (id_provincia) REFERENCES provincias(id_provincia) ON DELETE RESTRICT ON UPDATE CASCADE,
    FOREIGN KEY (id_rango_edad) REFERENCES rango_edades(id_rango_edad) ON DELETE RESTRICT ON UPDATE CASCADE,
    FOREIGN KEY (id_nacionalidad) REFERENCES nacionalidades(id_nacionalidad) ON DELETE RESTRICT ON UPDATE CASCADE,
    FOREIGN KEY (id_sexo) REFERENCES sexos(id_sexo) ON DELETE RESTRICT ON UPDATE CASCADE,
    FOREIGN KEY (id_anio) REFERENCES anios(id_anio) ON DELETE RESTRICT ON UPDATE CASCADE
    
);
"""
cursor.execute(query_creacion)   
conexion.commit()

In [155]:
lista_de_tuplas=[]
for fila in df_poblacional.values:
    lista_de_tuplas.append(tuple(fila))


In [157]:
lista_de_tuplas = [(int(x),float(a),int(b),int(c),int(w),int(p),int(k))for x,a,b,c,w,p,k in lista_de_tuplas]


In [159]:
     
query_insercion= "INSERT INTO poblacionales (id_poblacion,Total,id_provincia,id_rango_edad,id_nacionalidad,id_sexo,id_anio) VALUES (%s, %s, %s, %s, %s, %s, %s);"     

cursor.executemany(query_insercion, lista_de_tuplas)          
conexion.commit()
cursor.close()
conexion.close()